**This notebook is an exercise in the [Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/data-leakage).**

---


Most people find target leakage very tricky until they've thought about it for a long time.

So, before trying to think about leakage in the housing price example, we'll go through a few examples in other applications. Things will feel more familiar once you come back to a question about house prices.

# Setup

The questions below will give you feedback on your answers. Run the following cell to set up the feedback system.

In [1]:
# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex7 import *
print("Setup Complete")

Setup Complete


**Data Leakage** : 학습 데이터가 타겟에 대한 정보를 가지고 있지만, 그 정보들을 실제 예측에 사용할 수 없는 경우 발생한다. 학습 데이터, 심지어 validation 데이터에 대해서도 학습이 잘 되었지만 모델 배포후에는 제대로 동작하지 않는다는 의미이다. 

두 가지 data leakage가 존재: Target leakage 와 train-test contamination


1. Target Leakage

leakage는 예측 할 때 사용할 수 없는 데이터가 학습 데이터 셋에 포함되어 있을 때 발생한다.


특정 변수가 타겟 변수와 긴밀한 연관성을 보이는 것은 모델 학습에 좋은 영향을 끼칠 수 있다. 하지만 해당 변수가 의사 결정과정에서 문제가 있거나 구할 수 없는 데이터라면 예를 들어 가격 예측시 당일 원자재 값은 물건의 가격에 긴밀한 관계가 있지만 예측을 요하는 문제에서 당일 자재값은 구할 수 없다.

따라서 해당 피쳐가 예측 시점에 사용이 될 수 있을지 생각해보고, 그렇지 않다면 데이터셋에서 제외시켜야 한다.

 

2. Train-Test Contamination

이 data leakage는 학습 데이터와 validation 데이터를 제대로 구분하지 않았을 때 생긴다.

 

Recall that validation는 모델이 이전에 고려되지 않았던 데이터에 대해 어떻게 작동하는지 측정하는 것을 말한다. Validation 데이터가 전처리에 영향을 준다면 이 과정에 손상이 올 수도 있다.

만약  train_test_split()을 전처리 과정(missing value를 처리하는 imputer 같은) 이전에 한다고 생각해보자. 결과는? Validation 스코어는 좋겠지만 배포 후의 성능은 별로일 것이다.

 

만약 validation 데이터가 train-test split을 기반으로 만들어졌을 때, validation 데이터를 모든 fitting에서 제외하고, 전처리 단계의 fitting에 포함시켜야 한다. Scikit-learn의 pipelines을 이용하면 더 쉽다. Cross-validation을 사용할 때는 파이프 라인 내에서 전처리를 수행하는 것이 훨씬 더 중요하다.

------------------------------------------
**문제점**
- 기계학습 비교시 문제가 됨
    - 상위 모델은 기본 문제의 좋은 일반화 된 모델이 아닌 누수 데이터를 사용
- 데이터를 제공하는 회사일 때 문제가 됨
    - 익명화 및 난독화를 되돌리는 것은 예상치 못한 개인 정보 침해로 이어질 가능성 존재
- 예측 모델을 개발할 때 문제가 됨   
    - 생산에 사용할 수 없는 낙관적인 모델을 만들수 있는 가능성 존재 (과적합)
    
    
**확인 방법**
- 과한 정확도가 나오는지 확인(과적합 확인)
- 복잡한 데이터에서 문제가 많이 발생
    - 시계열, 그래프, 사운드, 이미지 관측
    
**모델 학습시 Data Leakage를 최소화하는 방법**
- cross validation folds내에서 데이터 준비
    - 예측을 위한 데이터를 미리 정규화 표준화 시키면 Data Leakage 문제 발생 가능 (예측 데이터는 건들지 말기)
- 개발 된 모델의 최종 검사를 위한 validation set 보유
- 노이즈 추가
- 누수 변수 제거
- 파이프라인 사용

# Step 1: The Data Science of Shoelaces

Nike has hired you as a data science consultant to help them save money on shoe materials. Your first assignment is to review a model one of their employees built to predict how many shoelaces they'll need each month. The features going into the machine learning model include:
- The current month (January, February, etc)
- Advertising expenditures in the previous month
- Various macroeconomic features (like the unemployment rate) as of the beginning of the current month
- The amount of leather they ended up using in the current month

The results show the model is almost perfectly accurate if you include the feature about how much leather they used. But it is only moderately accurate if you leave that feature out. You realize this is because the amount of leather they use is a perfect indicator of how many shoes they produce, which in turn tells you how many shoelaces they need.

Do you think the _leather used_ feature constitutes a source of data leakage? If your answer is "it depends," what does it depend on?

After you have thought about your answer, check it against the solution below.

In [2]:
# Check your answer (Run this code cell to receive credit!)
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

This is tricky, and it depends on details of how data is collected (which is common when thinking about leakage). Would you at the beginning of the month decide how much leather will be used that month? If so, this is ok. But if that is determined during the month, you would not have access to it when you make the prediction. If you have a guess at the beginning of the month, and it is subsequently changed during the month, the actual amount used during the month cannot be used as a feature (because it causes leakage).

1. 신발 끈의 데이터 과학

나이키는 당신을 신발 재료의 비용을 절감을 위해 데이터 사이언스 컨설턴트로 고용하였습니다. 첫 번째 과제는 직원 중 한 명이 매달 얼마나 많은 신발끈이 필요할지 예측하기 위해 만든 모델을 검토하는 것입니다. 기계 학습 모델에 포함되는 기능은 다음과 같습니다.

- 현재 월(1월,2월 등)
- 전월의 광고지출
- 이달 초 현재 다양한 거시경제적 특성(예: 실업률)
- 이번 달에 사용된 가죽의 양

가죽 사용량에 대한 피처들을 포함하면 모델은 거의 완벽하게 정확하다고 보여집니다. 그러나 이 피처는 생략하면 중간 정도로 정확합니다. 그들이 사용하는 가죽의 양이 얼마나 많은 신발을 생산하는지에 대한 완벽한 지표이기 때문이라는 것을 알게 됩니다.

가죽 사용량 피처가 데이터 누수의 원인이 된다고 생각하십니까? 만약 당신의 대답이 "그것은 아니다"라면, 그것은 무엇에 의존합니까?

답변에 대해 생각해 본 후 아래 솔루션과 비교해 보십시오.

**ans) 이는 까다롭고 데이터 수집 방법에 따라 다릅니다(누수을 고려할 때 흔히 있음). 그 달에 가죽을 얼마나 사용할지 월초에 결정해 주시겠습니까? 그렇다면 이 정도면 됩니다. 그러나 월 중에 이 값이 결정되면 예측에 액세스할 수 없습니다. 월초에 대한 예측이 있고 그 후에 월 중에 변경되는 경우 월 중에 사용한 실제 양은 피쳐로 사용할 수 없습니다(누수을 유발하므로).**

# Step 2: Return of the Shoelaces

You have a new idea. You could use the amount of leather Nike ordered (rather than the amount they actually used) leading up to a given month as a predictor in your shoelace model.

Does this change your answer about whether there is a leakage problem? If you answer "it depends," what does it depend on?

In [3]:
# Check your answer (Run this code cell to receive credit!)
q_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

This could be fine, but it depends on whether they order shoelaces first or leather first. If they order shoelaces first, you won't know how much leather they've ordered when you predict their shoelace needs. If they order leather first, then you'll have that number available when you place your shoelace order, and you should be ok.

2. 신발끈 환불

새로운 아이디어가 떠올랐군요 신발끈 모델의 예측 변수는 나이키가 주문한 가죽의 양(실제 사용한 양보다)을 최대 한 달까지 사용할 수 있습니다.

이렇게 하면 누수 문제가 있는지 여부에 대한 답변이 달라집니까? 만약 당신이 "그것이 아니다"라고 대답한다면, 그것은 무엇에 달려있나요?

**ans) 이 정도면 괜찮겠지만, 신발끈을 먼저 주문하느냐 가죽을 먼저 주문하느냐에 따라 달라요. 신발끈을 먼저 주문하면 신발끈이 필요할 것으로 예상했을 때 가죽을 얼마나 주문했는지 알 수 없을 거예요. 가죽을 먼저 주문하면 신발끈 주문 시 이용가능한 수가 나오는데 괜찮을 거예요.**

# 3. Getting Rich With Cryptocurrencies?

You saved Nike so much money that they gave you a bonus. Congratulations.

Your friend, who is also a data scientist, says he has built a model that will let you turn your bonus into millions of dollars. Specifically, his model predicts the price of a new cryptocurrency (like Bitcoin, but a newer one) one day ahead of the moment of prediction. His plan is to purchase the cryptocurrency whenever the model says the price of the currency (in dollars) is about to go up.

The most important features in his model are:
- Current price of the currency
- Amount of the currency sold in the last 24 hours
- Change in the currency price in the last 24 hours
- Change in the currency price in the last 1 hour
- Number of new tweets in the last 24 hours that mention the currency

The value of the cryptocurrency in dollars has fluctuated up and down by over $\$$100 in the last year, and yet his model's average error is less than $\$$1. He says this is proof his model is accurate, and you should invest with him, buying the currency whenever the model says it is about to go up.

Is he right? If there is a problem with his model, what is it?

In [4]:
# Check your answer (Run this code cell to receive credit!)
q_3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

There is no source of leakage here. These features should be available at the moment you want to make a predition, and they're unlikely to be changed in the training data after the prediction target is determined. But, the way he describes accuracy could be misleading if you aren't careful. If the price moves gradually, today's price will be an accurate predictor of tomorrow's price, but it may not tell you whether it's a good time to invest. For instance, if it is $100 today, a model predicting a price of $100 tomorrow may seem accurate, even if it can't tell you whether the price is going up or down from the current price. A better prediction target would be the change in price over the next day. If you can consistently predict whether the price is about to go up or down (and by how much), you may have a winning investment opportunity.

3. 가상화폐로 부자가 되시겠습니까?

당신은 나이키를 너무 많이 저축해서 그들이 당신에게 보너스를 주었어요. 축하해요.

데이터 과학자이기도 한 여러분의 친구는 여러분의 보너스를 수백만 달러로 바꿀 수 있는 모델을 만들었다고 말합니다. 구체적으로 그의 모델은 예측 순간보다 하루 앞서 새로운 암호화 화폐(예: 비트코인, 그러나 새로운 암호화 화폐)의 가격을 예측합니다. 통화(달러 단위) 가격이 오르기 직전이라고 모델이 말할 때마다 이 암호화를 구입하는게 그의 계획입니다.

그의 모델에서 가장 중요한 피처은 다음과 같습니다.

통화의 현재 가격:
- 최근 24시간 동안 판매된 통화의 양
- 최근 24시간 동안의 통화 가격 변동
- 최근 1시간 동안의 통화 가격 변동
- 최근 24시간 동안 통화를 언급한 새 트윗 수

달러화 가상화폐 가치는 지난해 100달러 이상 오르락내리락했지만 그의 모델 평균 오차는 1달러 미만입니다. 그는 이것이 자신의 모델이 정확하다는 증거이며, 모델이 곧 오를 것이라고 말할 때마다 화폐를 사면서 함께 투자해야 한다고 말합니다.

그가 옳나요? 그의 모델에 문제가 있다면 무엇입니까?


**ans) 여기에는 누수의 원인이 없습니다. 이러한 기능은 예측하려는 시점에 사용할 수 있어야 하며 예측 대상이 결정된 후에도 train 데이터에서 변경되지 않을 가능성이 높습니다. 하지만 조심하지 않으면 정확성을 오인할 수 있습니다. 가격이 서서히 움직인다면 오늘의 가격이 내일의 가격을 정확하게 예측할 수 있을 것이지만 지금이 투자하기에 좋은 시기인지 여부는 알려주지 않을 수도 있습니다. 예를 들어 오늘 가격이 100달러라면 현재 가격보다 오르는지 내리는지 알 수 없더라도 모델 예측값인 100달러는 내일 가격인 100달러와 비교했을때 정확해 보일 수 있습니다. 더 나은 예측 목표는 다음 날의 가격 변동입니다. 가격이 상승할지 하강할지(얼마인지)를 일관되게 예측할 수 있다면 투자 기회를 확보할 수 있습니다.**

# Step 4: Preventing Infections

An agency that provides healthcare wants to predict which patients from a rare surgery are at risk of infection, so it can alert the nurses to be especially careful when following up with those patients.

You want to build a model. Each row in the modeling dataset will be a single patient who received the surgery, and the prediction target will be whether they got an infection.

Some surgeons may do the procedure in a manner that raises or lowers the risk of infection. But how can you best incorporate the surgeon information into the model?

You have a clever idea. 
1. Take all surgeries by each surgeon and calculate the infection rate among those surgeons.
2. For each patient in the data, find out who the surgeon was and plug in that surgeon's average infection rate as a feature.

Does this pose any target leakage issues?
Does it pose any train-test contamination issues?

In [5]:
# Check your answer (Run this code cell to receive credit!)
q_4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

This poses a risk of both target leakage and train-test contamination (though you may be able to avoid both if you are careful).

You have target leakage if a given patient's outcome contributes to the infection rate for his surgeon, which is then plugged back into the prediction model for whether that patient becomes infected. You can avoid target leakage if you calculate the surgeon's infection rate by using only the surgeries before the patient we are predicting for. Calculating this for each surgery in your training data may be a little tricky.

You also have a train-test contamination problem if you calculate this using all surgeries a surgeon performed, including those from the test-set. The result would be that your model could look very accurate on the test set, even if it wouldn't generalize well to new patients after the model is deployed. This would happen because the surgeon-risk feature accounts for data in the test set. Test sets exist to estimate how the model will do when seeing new data. So this contamination defeats the purpose of the test set.

4. 감염을 방지

의료 서비스를 제공하는 한 기관은 희귀 수술에서 어떤 환자가 감염 위험에 있는지 예측하고자하여 간호사들에게 환자를 확인할 때 특히 주의하도록 경고할 수 있습니다.

당신은 모델을 구축하려고 합니다. 모델링 데이터 세트의 각 행은 수술을 받은 단일 환자이며, 예측 대상은 감염 여부입니다.

일부 의사들은 감염의 위험을 높이거나 낮추는 방식으로 시술할 수 있습니다. 하지만 어떻게 외과의사 정보를 모델에 가장 잘 반영할 수 있을까요?

좋은 생각이 있습니다.

1. 각 외과의사의 모든 수술을 받고 그 외과의들의 감염률을 계산하세요.
2. 데이터에 있는 각 환자에 대해 외과의사가 누구인지 알아보고 해당 외과의사의 평균 감염률을 특징으로 꼽습니다.

대상 누수 문제가 있습니까? 훈련-테스트 오염 문제가 있습니까?

**ans) 이는 타겟 누수 및 훈련-테스트 오염의 위험을 모두 초래할 수 있습니다(단, 주의할 경우 두 가지 모두를 피할 수 있음).**

**주어진 환자의 결과가 외과의사의 감염률에 기여하는 경우 훈련-테스트 오염이 발생합니다. 그러면 해당 환자의 감염 여부를 예측 모델에 다시 연결됩니다. 우리가 예상하는 환자의 이전 수술만 사용하여 외과의사의 감염률을 계산하면 타겟 누수 문제를 피할 수 있습니다. 교육 데이터의 각 수술에 대해 이 값을 계산하는 것은 다소 까다로울 수 있습니다.**

**또한 검사 세트의 수술을 포함하여 의사가 수행한 모든 수술을 사용하여 이 값을 계산하면 훈련-테스트 오염 문제가 발생합니다. 따라서 모델이 배포된 후 새 환자에게 잘 일반화되지 않더라도 테스트 세트에서 모델이 매우 정확하게 보일 수 있습니다. 이 문제는 수술-위험 피처가 테스트 세트의 데이터를 설명하기 때문에 발생합니다. 검정 세트는 새 데이터를 볼 때 모형이 어떻게 수행할지 추정하기 위해 존재합니다. 따라서 이 오염은 테스트 세트의 목적을 무시합니다.**

# Step 5: Housing Prices

You will build a model to predict housing prices.  The model will be deployed on an ongoing basis, to predict the price of a new house when a description is added to a website.  Here are four features that could be used as predictors.
1. Size of the house (in square meters)
2. Average sales price of homes in the same neighborhood
3. Latitude and longitude of the house
4. Whether the house has a basement

You have historic data to train and validate the model.

Which of the features is most likely to be a source of leakage?

In [6]:
# Fill in the line below with one of 1, 2, 3 or 4.
potential_leakage_feature = 2

# Check your answer
q_5.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

2 is the source of target leakage. Here is an analysis for each feature: 

1. The size of a house is unlikely to be changed after it is sold (though technically it's possible). But typically this will be available when we need to make a prediction, and the data won't be modified after the home is sold. So it is pretty safe. 

2. We don't know the rules for when this is updated. If the field is updated in the raw data after a home was sold, and the home's sale is used to calculate the average, this constitutes a case of target leakage. At an extreme, if only one home is sold in the neighborhood, and it is the home we are trying to predict, then the average will be exactly equal to the value we are trying to predict.  In general, for neighborhoods with few sales, the model will perform very well on the training data.  But when you apply the model, the home you are predicting won't have been sold yet, so this feature won't work the same as it did in the training data. 

3. These don't change, and will be available at the time we want to make a prediction. So there's no risk of target leakage here. 

4. This also doesn't change, and it is available at the time we want to make a prediction. So there's no risk of target leakage here.

In [7]:
q_5.hint()
q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Which of these features might be updated in a database after the house is     sold? That's the one to worry about.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 2 is the source of target leakage. Here is an analysis for each feature: 

1. The size of a house is unlikely to be changed after it is sold (though technically it's possible). But typically this will be available when we need to make a prediction, and the data won't be modified after the home is sold. So it is pretty safe. 

2. We don't know the rules for when this is updated. If the field is updated in the raw data after a home was sold, and the home's sale is used to calculate the average, this constitutes a case of target leakage. At an extreme, if only one home is sold in the neighborhood, and it is the home we are trying to predict, then the average will be exactly equal to the value we are trying to predict.  In general, for neighborhoods with few sales, the model will perform very well on the training data.  But when you apply the model, the home you are predicting won't have been sold yet, so this feature won't work the same as it did in the training data. 

3. These don't change, and will be available at the time we want to make a prediction. So there's no risk of target leakage here. 

4. This also doesn't change, and it is available at the time we want to make a prediction. So there's no risk of target leakage here.

5. 주택 가격
당신은 주택 가격을 예측하는 모델을 만들 것입니다. 이 모델은 웹사이트에 설명이 추가될 때 새로운 주택의 가격을 예측하기 위해 지속적으로 배치될 것입니다. 다음은 예측 변수로 사용할 수 있는 네 가지 피처입니다.

- 집 크기(제곱미터 단위)
- 동일 지역 주택의 평균 판매 가격
- 집의 위도 및 경도
- 집의 지하실 존재 여부

모델을 교육하고 검증할 과거 데이터가 있습니다.

누출의 원인이 될 가능성이 가장 높은 기능은 무엇입니까?


**ans) 동일 지역 주택의 평균 판매 가격는 타겟 누수의 원인입니다. 다음은 각 피쳐에 대한 분석입니다.**

**집 크기(제곱미터 단위) : 집이 팔리고 나면 크기가 바뀔 것 같지 않습니다(기술적으로는 가능하지만). 그러나 일반적으로 이것은 우리가 예측해야 할 때 사용할 수 있을 것이고, 주택이 팔린 후에도 데이터는 수정되지 않을 것입니다. 그래서 꽤 안전합니다.**

**동일 지역 주택의 평균 판매 가격 : 업데이트 시 규칙을 알 수 없습니다. 주택이 매매된 후 원시 데이터에서 필드를 업데이트하고 주택 매각을 평균 계산에 사용하는 경우, 이는 타겟 누수에 해당합니다. 극단적으로, 이웃에 단 한 채의 집만 팔리고, 우리가 예측하려는 집이라면, 평균은 우리가 예측하려는 값와 정확히 같을 것입니다. 일반적으로 판매량이 적은 동네의 경우 이 모형은 교육 데이터에 대해 매우 우수한 성능을 발휘합니다. 하지만 모델을 적용하면 예상했던 집이 아직 팔리지 않아 이 피처은 교육 데이터와 동일하게 작동하지 않을 것입니다.**

**집의 위도 및 경도 : 이 피처은 변경되지 않으며 예측하려는 시점에 사용할 수 있습니다. 따라서 여기에는 타겟이 누수될 위험이 없습니다.**

**집의 지하실 존재 여부 : 이 또한 변경되지 않으며, 예측하려는 시점에 사용할 수 있습니다. 따라서 타겟이 누수될 위험이 없습니다.**

# Conclusion
Leakage is a hard and subtle issue. You should be proud if you picked up on the issues in these examples.

Now you have the tools to make highly accurate models, and pick up on the most difficult practical problems that arise with applying these models to solve real problems.

There is still a lot of room to build knowledge and experience. Try out a [Competition](https://www.kaggle.com/competitions) or look through our [Datasets](https://kaggle.com/datasets) to practice your new skills.

Again, Congratulations!

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161289) to chat with other Learners.*